<a href="https://colab.research.google.com/github/ZinTun/TopicModelling/blob/main/TopicModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Twitter API

In [ ]:
import tweepy
import pandas as pd

####input your credentials here
consumer_key = 'xxx'
consumer_secret = 'xxx'
access_token = 'xxx'
access_token_secret = 'xxx'

####
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

tweet_list=[]
#### getting tweets with #Feb19Coup
tweets = tweepy.Cursor(api.search, q="#Feb19Coup", lang = "en", tweet_mode='extended').items()
for tweet_info in tweets:
  ### the full_text retrieval for retweet and normal tweets are not the same
  if 'retweeted_status' in dir(tweet_info):
      tweet_list.append(tweet_info.retweeted_status.full_text)
  else:
      tweet_list.append(tweet_info.full_text)

Data Cleaning

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer 
import string

### lower case 
doc_lowercase = [doc.lower() for doc in tweet_list]

### remove links
doc_nolinks = [re.sub('http\S+', ' ', doc) for doc in doc_lowercase]

## remove \n
doc_wolinefeed = [re.sub('\s+', ' ', doc) for doc in doc_nolinks]

## remove @
doc_noemail = [re.sub('\S*@\S*\s?', '', doc) for doc in doc_wolinefeed]

## remove emoji
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
                           
doc_noemoji = [emoji_pattern.sub(r'', doc) for doc in doc_noemail]

stop_words = stopwords.words('english')
## adding additional stop-words after analyzing the original corpus
stop_words.extend(['#whatshappeninginmyanmar', '#feb19coup', "us", "u"])
exclude = set(string.punctuation)
ps = PorterStemmer() 

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop_words])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    stemmed = " ".join( ps.stem(word) for word in punc_free.split())
    return stemmed

doc_clean = [clean(doc).split() for doc in doc_noemoji] 

LDA Model Training

In [ ]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary 
mydict = corpora.Dictionary(doc_clean)

# Converting into Document Term Matrix 
doc_term_matrix = [mydict.doc2bow(doc,allow_update=True) for doc in doc_clean]
print(doc_term_matrix)

# Create LDA object using gensim library
Lda = gensim.models.ldamodel.LdaModel

# train LDA model
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = mydict, passes=50)

#save model
ldamodel.save('lda_model.model')

ldamodel.print_topics(-1)
# [(0,
#   '0.041*"thwate" + 0.035*"peac" + 0.026*"mya" + 0.025*"junta" + 0.025*"head" + 0.024*"polic" + 0.024*"shot" + 0.024*"demonstr" + 0.023*"kha" + 0.022*"bullet"'),
#  (1,
#   '0.045*"need" + 0.042*"justic" + 0.034*"protest" + 0.030*"myanmar" + 0.027*"myitkyina" + 0.022*"militari" + 0.019*"peac" + 0.014*"support" + 0.014*"whatshappeninglnmyanmar" + 0.012*"democraci"'),
#  (2,
#   '0.034*"myitkyina" + 0.030*"teacher" + 0.030*"arrest" + 0.026*"peopl" + 0.023*"polic" + 0.022*"militari" + 0.022*"two" + 0.018*"want" + 0.017*"need" + 0.016*"justic"')]

Visualization

In [ ]:
topics=ldamodel.show_topics(num_topics=3, num_words=20,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in topics]

def drawwordcloud(filename):
  wordcloud = WordCloud(max_font_size=200, max_words=75, font_step=2, width= 800, height=400, background_color="white").generate(text)
  wordcloud.to_file(filename)

  plt.figure(dpi=300)
  plt.imshow(wordcloud, interpolation="bilinear")
  plt.axis("off")
  plt.show()


count = 1;
for topic,words in topics_words:
    text = " ".join(words)
    drawwordcloud("topic"+str(count)+".png")
    count = count +1